In [ ]:
import os

import h5py
import numpy as np
from scipy.ndimage.interpolation import zoom

In [ ]:
data_root = '/home/yutongx/local_data/'

src_root = os.path.join(data_root, 'preprocessed_3d')
src_train_path = os.path.join(src_root, 'train')
src_val_path = os.path.join(src_root, 'val')

dst_root = os.path.join(data_root, 'preprocessed_3d_128')
dst_train_path = os.path.join(dst_root, 'train')
dst_val_path = os.path.join(dst_root, 'val')

In [ ]:
def downsampling(src_path, dst_path, order, gamma=None, median=None):
    src_names = sorted(os.listdir(src_path))
    
    for src_name in src_names:
        file_src = h5py.File(os.path.join(src_path, src_name),'r')
        img = file_src['raw'][:]
        lbl = file_src['label'][:]
        img_shape = file_src['ori_shape'][:]
        img_affine = file_src['ori_affine'][:]
        file_src.close()
        
        #gamma transformation, optional, use value smaller than 1
        if gamma is not None:
            img = img**(gamma)
        #median filter, optional
        if median is True:
            img = cv2.medianBlur(img, 3)
        
        img_resampled = (zoom(img, (0.5,1,1), order=order, mode='nearest')).astype(np.float32)
        lbl_resampled = (zoom(lbl, (0.5,1,1), order=order, mode='nearest')).astype(np.uint8)

        file_dst = h5py.File(os.path.join(dst_path, src_name),'w')
        file_dst.create_dataset('raw', data=img_resampled)
        file_dst.create_dataset('label', data=lbl_resampled)
        file.create_dataset('ori_shape', data=img_shape)
        file_dst.create_dataset('ori_affine', data=img_affine)
        file_dst.close()

In [ ]:
downsampling(src_train_path, dst_train_path, order=0)

In [ ]:
downsampling(src_val_path, dst_val_path, order=0)